In [1]:
import pandas as pd

In [2]:
data = pd.read_csv(r'/content/total_features.csv')

In [3]:
val = pd.read_csv(r'/content/val_last.csv')

In [4]:
def remove_random_share(df, share):
    """
    Удаляет случайную долю строк из DataFrame, где y == 0.25.

    :param df: DataFrame, из которого нужно удалить строки
    :param share: Доля строк для удаления (например, 0.1 для 10%)
    :return: Очищенный DataFrame
    """
    # Фильтруем строки, где y == 0.25
    rows_to_remove = df.query('y ==0.25 and recommended_algo1 ==0 and recommended_algo2 ==0')

    # Определяем количество строк для удаления
    num_rows_to_remove = int(len(rows_to_remove) * share)

    # Проверяем, что количество строк для удаления не превышает доступное количество
    if num_rows_to_remove > 0:
        # Случайным образом выбираем строки для удаления
        rows_to_remove_indices = rows_to_remove.sample(n=num_rows_to_remove, random_state=42).index

        # Удаляем выбранные строки из оригинального DataFrame
        df_cleaned = df.drop(index=rows_to_remove_indices)
    else:
        # Если нет строк для удаления, возвращаем оригинальный DataFrame
        df_cleaned = df.copy()

    return df_cleaned

In [5]:
# val = remove_random_share(val, 0.6)
val = val[val.y != 0.25]

In [6]:
client_features = data.groupby(by = 'user_id')[[ 'user_rating_count', 'user_genre0_share', 'user_genre1_share',
    'user_genre2_share', 'user_genre3_share', 'user_genre4_share',
    'user_genre5_share', 'user_genre6_share', 'user_genre7_share',
    'user_genre8_share', 'user_genre9_share', 'user_genre10_share',
    'user_genre11_share', 'user_genre12_share', 'user_genre13_share',
    'user_genre14_share', 'user_genre15_share', 'user_genre16_share',
    'user_genre17_share', 'user_timestamp_q0', 'user_timestamp_q10',
    'user_timestamp_q25', 'user_timestamp_q33', 'user_timestamp_q50',
    'user_timestamp_q67', 'user_timestamp_q75', 'user_timestamp_q90',
    'user_timestamp_q100', 'user_timestamp_range', 'user_timestamp_iqr',
     'age']].mean().reset_index()

In [7]:
item_features = data.groupby(by = 'item_id')[['female_ratio', 'male_ratio',
    'young_ratio', 'adult_ratio', 'senior_ratio',
    'female_ratio_genre', 'male_ratio_genre', 'young_ratio_genre',
    'adult_ratio_genre', 'senior_ratio_genre', 'item_rating_count',
    'avg_rating_time_x', 'rating_time_range_x',
    'genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5',
    'genre_6', 'genre_7', 'genre_8', 'genre_9', 'genre_10', 'genre_11',
    'genre_12', 'genre_13', 'genre_14', 'genre_15', 'genre_16', 'genre_17']].mean().reset_index()

In [8]:
import numpy as np
parts = np.array_split(val, 5)
merged_results = []

# Цикл для объединения каждой части
for part in parts:
    # Выполняем left join с client_features по user_id
    val_with_client_features = part.merge(client_features, on='user_id', how='left')

    # Выполняем left join с item_features по item_id
    val_final_merged = val_with_client_features.merge(item_features, on='item_id', how='left')

    # Добавляем результат в список
    merged_results.append(val_final_merged)
    del val_with_client_features
    del val_final_merged


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [9]:
del client_features
del item_features
del parts
del part

In [10]:
val = pd.concat(merged_results)

In [11]:
del merged_results

In [12]:
# val = pd.read_csv('/kaggle/input/val-last/val_last.csv')

In [13]:
len(val.columns)

67

In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV
! pip install catboost
from catboost import CatBoostRegressor

In [15]:
X = val.drop(columns=['y'])  # Все столбцы, кроме 'y'
y = val['y']  # Целевая переменная

In [16]:
y.dtypes

dtype('float64')

In [18]:
y.replace(0.5, 0.7, inplace=True)

In [19]:
del val

In [17]:

# Разделяем данные на обучающую и тестовую выборки со стратификацией
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [21]:
# del X
# del y

In [ ]:

# Разделяем данные на обучающую и тестовую выборки со стратификацией
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Определяем модель с использованием GPU
model = CatBoostRegressor(verbose=1000, ignored_features = ['user_id', 'item_id'], task_type='GPU')  # Устанавливаем verbose=0, чтобы не выводить информацию во время обучения

# Определяем гиперпараметры для поиска
param_grid = {
    'iterations': [1500, 2000],
    'learning_rate': [0.1],
    'depth': [12, 14, 16]
}

# Создаем объект GridSearchCV
grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',  # Используем MSE в качестве метрики
                           cv=3,  # Кросс-валидация на 3 фолда
                           verbose=1000)  # Используем все доступные ядра

# Обучение модели с поиском по сетке
grid_search.fit(X_train, y_train)

# Получаем лучшие параметры и модель
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Лучшие параметры:", best_params)

# Оценка модели на тестовой выборке
test_score = best_model.score(X_test, y_test)
print("Тестовый балл:", test_score)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3; 1/6] START depth=12, iterations=1500, learning_rate=0.1................
0:	learn: 0.3888810	total: 118ms	remaining: 2m 56s
1000:	learn: 0.1377340	total: 40.2s	remaining: 20s
1499:	learn: 0.0976969	total: 1m 1s	remaining: 0us
[CV 1/3; 1/6] END depth=12, iterations=1500, learning_rate=0.1;, score=-0.104 total time= 1.1min
[CV 2/3; 1/6] START depth=12, iterations=1500, learning_rate=0.1................
0:	learn: 0.3893201	total: 56.1ms	remaining: 1m 24s
1000:	learn: 0.1373357	total: 36.7s	remaining: 18.3s
1499:	learn: 0.0971020	total: 55.2s	remaining: 0us
[CV 2/3; 1/6] END depth=12, iterations=1500, learning_rate=0.1;, score=-0.103 total time=  57.7s
[CV 3/3; 1/6] START depth=12, iterations=1500, learning_rate=0.1................
0:	learn: 0.3892490	total: 25.9ms	remaining: 38.8s
1000:	learn: 0.1357764	total: 38.5s	remaining: 19.2s
1499:	learn: 0.0959183	total: 58.2s	remaining: 0us
[CV 3/3; 1/6] END depth=12, iterations=

In [56]:
model = best_model

In [57]:
X = pd.concat([X_train, X_test])

In [58]:
y = pd.concat([y_train, y_test])

In [59]:
model.fit(X, y)

0:	learn: 0.3890697	total: 113ms	remaining: 2m 49s
1000:	learn: 0.1941061	total: 47.1s	remaining: 23.5s
1499:	learn: 0.1592250	total: 1m 7s	remaining: 0us


In [43]:
import joblib
joblib.dump(model, r'/content/model.joblib')

['/content/model.joblib']

In [60]:
import pandas as pd
data = pd.read_csv(r'/content/total_features.csv')
client_features = data.groupby(by = 'user_id')[[ 'user_rating_count', 'user_genre0_share', 'user_genre1_share',
    'user_genre2_share', 'user_genre3_share', 'user_genre4_share',
    'user_genre5_share', 'user_genre6_share', 'user_genre7_share',
    'user_genre8_share', 'user_genre9_share', 'user_genre10_share',
    'user_genre11_share', 'user_genre12_share', 'user_genre13_share',
    'user_genre14_share', 'user_genre15_share', 'user_genre16_share',
    'user_genre17_share', 'user_timestamp_q0', 'user_timestamp_q10',
    'user_timestamp_q25', 'user_timestamp_q33', 'user_timestamp_q50',
    'user_timestamp_q67', 'user_timestamp_q75', 'user_timestamp_q90',
    'user_timestamp_q100', 'user_timestamp_range', 'user_timestamp_iqr',
     'age']].mean().reset_index()

item_features = data.groupby(by = 'item_id')[['female_ratio', 'male_ratio',
    'young_ratio', 'adult_ratio', 'senior_ratio',
    'female_ratio_genre', 'male_ratio_genre', 'young_ratio_genre',
    'adult_ratio_genre', 'senior_ratio_genre', 'item_rating_count',
    'avg_rating_time_x', 'rating_time_range_x',
    'genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5',
    'genre_6', 'genre_7', 'genre_8', 'genre_9', 'genre_10', 'genre_11',
    'genre_12', 'genre_13', 'genre_14', 'genre_15', 'genre_16', 'genre_17']].mean().reset_index()

In [61]:
val_to_inference = pd.read_csv(r'/content/val_to_inference.csv')

In [62]:
val_to_inference.columns

Index(['user_id', 'item_id', 'recommended_algo1', 'recommended_algo2'], dtype='object')

In [63]:
val_to_inference = val_to_inference.query('recommended_algo1 == 1 or recommended_algo2 == 1')

In [64]:
len(val_to_inference.user_id.unique())

6040

In [65]:
import numpy as np
parts = np.array_split(val_to_inference, 5)
merged_results = []

# Цикл для объединения каждой части
for part in parts:
    # Выполняем left join с client_features по user_id
    val_with_client_features = part.merge(client_features, on='user_id', how='left')

    # Выполняем left join с item_features по item_id
    val_final_merged = val_with_client_features.merge(item_features, on='item_id', how='left')

    # Добавляем результат в список
    merged_results.append(val_final_merged)
    del val_with_client_features
    del val_final_merged

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [66]:
val_to_inference = pd.concat(merged_results)

In [51]:
import joblib
model = joblib.load(r'/content/model.joblib')

In [67]:
val_to_inference['boosting_forecast'] = model.predict(val_to_inference)

In [68]:
def get_submission_sample(before_range):
    top_items = (before_range
                .sort_values(by='boosting_forecast', ascending=False)
                .groupby('user_id')
                .head(10))

    # Теперь создаем новый датафрейм с user_id и строкой из item_id
    result = (top_items
            .groupby('user_id')['item_id']
            .apply(lambda x: ' '.join(x.astype(str)))
            .reset_index())

    # Переименуем столбцы для ясности
    result.columns = ['user_id', 'item_id']

    # Теперь result содержит user_id и соответствующие топ-10 item_id
    return result

In [69]:
val_to_inference = val_to_inference[['user_id', 'item_id', 'boosting_forecast']]

In [70]:
res = get_submission_sample(val_to_inference)

In [72]:
res.to_csv('submission_sample_new.csv', sep = ',', index = False)